#### Writing and Reading Time Series with TimeDB SDK

This notebook demonstrates:
1. Writing time series data without specifying series_id (auto-generated)
2. Reading data back and plotting with series_id in legend
3. Writing multiple time series with the same series_id (updates)
4. Reading back to see consolidated time series


In [1]:
import timedb as td
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
load_dotenv()

False

## Part 1: Two Different Time Series (Different Series IDs)

First, let's create the schema and write two different time series without specifying series_id.


In [2]:
# Delete database schema
td.delete()

In [3]:
# Create database schema
td.create()

Creating database schema...
✓ Schema created successfully


In [4]:
# Create first time series: Temperature data (first 24 hours: 0-23)
base_time = datetime(2025, 1, 1, 0, 0, tzinfo=timezone.utc)
dates1 = [base_time + timedelta(hours=i) for i in range(24)]
df1 = pd.DataFrame({
    'valid_time': dates1,
    'value': [20.0 + i * 0.3 for i in range(24)]  # Temperature rising
})

# Insert first time series (series_id will be auto-generated)
result1 = td.insert_run(df=df1)
print(result1)
print(f"✓ Inserted first time series with series_id: {result1.series_ids}")
print(f"  Time range: {dates1[0]} to {dates1[-1]}")


Data values inserted successfully.
InsertResult(run_id=UUID('5322a4b0-32c1-42ae-8f17-8ff62b989437'), workflow_id='sdk-workflow', series_ids={'value': UUID('42a7a84f-72fa-4d8b-81ff-054cc8b98ee6')}, tenant_id=UUID('00000000-0000-0000-0000-000000000000'))
✓ Inserted first time series with series_id: {'value': UUID('42a7a84f-72fa-4d8b-81ff-054cc8b98ee6')}
  Time range: 2025-01-01 00:00:00+00:00 to 2025-01-01 23:00:00+00:00


In [5]:
# Create second time series: Humidity data (next 24 hours: 24-47, following the first)
dates2 = [base_time + timedelta(hours=i) for i in range(24, 48)]  # Hours 24-47
df2 = pd.DataFrame({
    'valid_time': dates2,
    'value': [30.0 - (i-24) * 0.5 for i in range(24, 48)]  # Humidity decreasing
})

# Insert second time series (series_id will be auto-generated, different from first)
result2 = td.insert_run(df=df2)
print(f"✓ Inserted second time series with series_id: {result2.series_ids}")
print(f"  Time range: {dates2[0]} to {dates2[-1]}")


Data values inserted successfully.
✓ Inserted second time series with series_id: {'value': UUID('ed35bed3-477a-452c-b213-7c8c230ac86b')}
  Time range: 2025-01-02 00:00:00+00:00 to 2025-01-02 23:00:00+00:00


In [6]:
# Read back all time series
df_read = td.read()
# df_read.info()
# print(df_read.columns)
# print(df_read.columns.names)
# print(df_read.index)
print(f"✓ Read {len(df_read)} rows")
print(f"\nDataFrame shape: {df_read.shape}")


print(f"Index names: {df_read.index.names}")
print(f"Column level names: {df_read.columns.names}")
print(f"\nSeries keys in columns: {df_read.columns.get_level_values('series_key').unique()}")

print(f"\nSeries IDs in data: {df_read.index.get_level_values('series_ids').unique()}")
print(f"\nFirst few rows:")
print(df_read.head(10))


✓ Read 48 rows

DataFrame shape: (48, 2)
Index names: ['valid_time']
Column level names: ['series_key']

Series keys in columns: Index(['value'], dtype='object', name='series_key')


KeyError: 'Requested level (series_ids) does not match index name (valid_time)'

In [ ]:
# Prepare data for plotting
# Reset index to get series_id as a column
df_plot = df_read.reset_index()

# Handle potential duplicates: if same (valid_time, series_id, value_key) exists, take the first
# (This shouldn't happen with DISTINCT ON, but just in case)
df_plot = df_plot.drop_duplicates(subset=['valid_time', 'series_id', 'value_key'])

# Pivot to have series_id as columns for plotting
df_pivot = df_plot.pivot(index='valid_time', columns='series_id', values='value')

# Plot both time series
plt.figure(figsize=(12, 6))
for series_id in df_pivot.columns:
    plt.plot(df_pivot.index, df_pivot[series_id], marker='o', label=f'series_id: {series_id}')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Two Different Time Series (Different Series IDs)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"\n✓ Plotted {len(df_pivot.columns)} different time series (different series_ids)")


## Part 2: Same Series ID (Consolidated Time Series)

Now let's delete the schema and repeat, but this time we'll use the same series_id for both writes.


In [ ]:
# Delete schema to start fresh
td.delete()

# Create schema again
td.create()

In [ ]:
# Create a shared series_id
import uuid
shared_series_id = uuid.uuid4()
print(f"Using shared series_id: {shared_series_id}")


In [ ]:
# Insert first time series with explicit series_id
result1 = td.insert_run(df=df1, series_id=shared_series_id)
print(f"✓ Inserted first time series with series_id: {result1.series_id}")
print(f"  Time range: {dates1[0]} to {dates1[-1]}")

In [ ]:

# Insert second time series with the SAME series_id
result2 = td.insert_run(df=df2, series_id=shared_series_id)
print(f"✓ Inserted second time series with series_id: {result2.series_id}")
print(f"  Time range: {dates2[0]} to {dates2[-1]}")
print(f"  (Same series_id as first: {result1.series_id == result2.series_id})")
print(f"  Note: The two time series will be stitched together since they have consecutive time ranges!")


In [ ]:
# Read back all time series
df_read = td.read()
print(f"✓ Read {len(df_read)} rows")
print(f"\nDataFrame shape: {df_read.shape}")
print(f"\nSeries IDs in data: {df_read.index.get_level_values('series_id').unique()}")
print(f"\nFirst few rows:")
print(df_read.head(10))


In [ ]:
# Prepare data for plotting
# Reset index to get series_id as a column
df_plot = df_read.reset_index()

# Handle potential duplicates: if same (valid_time, series_id, value_key) exists, take the first
# (This shouldn't happen with DISTINCT ON, but just in case)
#f_plot = df_plot.drop_duplicates(subset=['valid_time', 'series_id', 'value_key'])

# Pivot to have series_id as columns for plotting
df_pivot = df_plot.pivot(index='valid_time', columns='series_id', values='value')

# Plot the time series
plt.figure(figsize=(12, 6))
for series_id in df_pivot.columns:
    plt.plot(df_pivot.index, df_pivot[series_id], marker='o', label=f'Series {series_id}')

plt.xlabel('Time')
plt.ylabel('Value')
plt.title('Time Series with Same Series ID (Consolidated)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"\n✓ Plotted {len(df_pivot.columns)} time series")
print(f"  Note: Both writes used the same series_id, so we see one consolidated time series")
